In [14]:
import fitz
import numpy as np
import pandas as pd

### import books

In [3]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

#### preset values (DPI)

In [4]:
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

### initialize sample page_num and book

In [9]:
page_num = 340
pages = vol1_pages

In [16]:
page = pages[page_num]

#### dev side notes

helpful documentation: https://pymupdf.readthedocs.io/en/latest/textpage.html#TextPage.extractDICT

<img src="https://pymupdf.readthedocs.io/en/latest/_images/img-textpage.png"
     alt="fitz documentation on get_text"
     style="float: left; margin-right: 10px; background-color: white;" />

<img src="./span_dict_doc.png"
     alt="fitz documentation on span blocks"
     style="float: left; margin-right: 10px; background-color: white;" />

In [40]:
# for each page, the heights and width values seem to be the same
pd.DataFrame(page.get_text("dict"))[['width', 'height']].value_counts()

width  height
426.0  642.0     59
dtype: int64

In [26]:
#the keys of each dictionary
page.get_text("dict")['blocks'][0].keys()

dict_keys(['number', 'type', 'bbox', 'lines'])

In [46]:
# TODO: I need to somehow connect the raw dict and dict variants
#       - possible way: see if a characters falls into the bbox of a word ... seems computationaly heavy

In [44]:
page.get_text("rawdict")['blocks'][0]

{'number': 0,
 'type': 0,
 'bbox': (170.63999938964844,
  22.124988555908203,
  402.01300048828125,
  31.024988174438477),
 'lines': [{'spans': [{'size': 8.855387687683105,
     'flags': 4,
     'font': 'Times-Roman',
     'color': 0,
     'ascender': 0.75,
     'descender': -0.25,
     'chars': [{'origin': (170.63999938964844, 28.79998779296875),
       'bbox': (170.63999938964844,
        22.124988555908203,
        177.00302124023438,
        31.024988174438477),
       'c': 'N'},
      {'origin': (176.675537109375, 28.79998779296875),
       'bbox': (176.675537109375,
        22.124988555908203,
        183.03855895996094,
        31.024988174438477),
       'c': 'O'},
      {'origin': (182.71107482910156, 28.79998779296875),
       'bbox': (182.71107482910156,
        22.124988555908203,
        189.0740966796875,
        31.024988174438477),
       'c': 'U'},
      {'origin': (188.74661254882812, 28.79998779296875),
       'bbox': (188.74661254882812,
        22.124988555908203,


In [45]:
page.get_text("dict")['blocks'][0]

{'number': 0,
 'type': 0,
 'bbox': (170.63999938964844,
  22.124988555908203,
  402.01300048828125,
  31.024988174438477),
 'lines': [{'spans': [{'size': 8.855387687683105,
     'flags': 4,
     'font': 'Times-Roman',
     'color': 0,
     'ascender': 0.75,
     'descender': -0.25,
     'text': 'NOUVELLE FLORE ',
     'origin': (170.63999938964844, 28.79998779296875),
     'bbox': (170.63999938964844,
      22.124988555908203,
      250.9449462890625,
      31.024988174438477)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (170.63999938964844,
    22.124988555908203,
    250.9449462890625,
    31.024988174438477)},
  {'spans': [{'size': 8.855387687683105,
     'flags': 4,
     'font': 'Times-Roman',
     'color': 0,
     'ascender': 0.75,
     'descender': -0.25,
     'text': '263',
     'origin': (387.25201416015625, 28.79998779296875),
     'bbox': (387.25201416015625,
      22.124988555908203,
      402.01300048828125,
      31.024988174438477)}],
   'wmode': 0,
   'dir': (1.0, 

In [71]:
text_blocks_SAMPLE = [block for block in page.get_text("dict")['blocks'] if block['type'] == 0]

In [72]:
#each text block -- as also confirmed by documentation -- has the following: 
text_blocks_SAMPLE[0].keys()

dict_keys(['number', 'type', 'bbox', 'lines'])

In [73]:
#each line -- as also confirmed by documentation -- has the following: 
text_blocks_SAMPLE[0]['lines'][0].keys()

dict_keys(['spans', 'wmode', 'dir', 'bbox'])

In [74]:
#each span ... 
text_blocks_SAMPLE[0]['lines'][0]['spans'][0].keys()

dict_keys(['size', 'flags', 'font', 'color', 'ascender', 'descender', 'text', 'origin', 'bbox'])

# Untitled

In [75]:
text_blocks = [block for block in page.get_text("dict")['blocks'] if block['type'] == 0]

In [79]:
#for each block in text blocks
for b_i in range(len(text_blocks)):
    b = text_blocks[b_i]
    curr_block_num_abs = b['number']                    #true blcok number
    curr_block_num_reletive = b_i                       #excludes image blocks
    curr_block_bbox = b['bbox']

    for l in b['lines']:
        curr_line_wmode = l['wmode']
        curr_line_dir = l['dir']
        curr_line_bbox = l['bbox']
        
        """the rest will be like the following 
        for s in span:
            for w in span:
                row_list.append({curr_block_num_abs ... etc to ... word...}
                can also do it with raw dict and say if white space then word = anything before the white space.
                -> Then add multiple rows each row being a character and word being word 
                   (only thing hard to get is bbox of the word ... not sure how important that is tho)
                   -- could also use max, mins of bbox of each character to find it

        ['spans', 'wmode', 'dir', 'bbox']"""

In [59]:
text_blocks

[{'number': 0,
  'type': 0,
  'bbox': (170.63999938964844,
   22.124988555908203,
   402.01300048828125,
   31.024988174438477),
  'lines': [{'spans': [{'size': 8.855387687683105,
      'flags': 4,
      'font': 'Times-Roman',
      'color': 0,
      'ascender': 0.75,
      'descender': -0.25,
      'text': 'NOUVELLE FLORE ',
      'origin': (170.63999938964844, 28.79998779296875),
      'bbox': (170.63999938964844,
       22.124988555908203,
       250.9449462890625,
       31.024988174438477)}],
    'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (170.63999938964844,
     22.124988555908203,
     250.9449462890625,
     31.024988174438477)},
   {'spans': [{'size': 8.855387687683105,
      'flags': 4,
      'font': 'Times-Roman',
      'color': 0,
      'ascender': 0.75,
      'descender': -0.25,
      'text': '263',
      'origin': (387.25201416015625, 28.79998779296875),
      'bbox': (387.25201416015625,
       22.124988555908203,
       402.01300048828125,
       31.024988174438477